In [ ]:
#If you have a GPU available, you can run the code using 
#using CuArrays
#gpu = true; CuArrays.allowscalar(false)

In [ ]:
#otherwise set gpu to false
gpu = false;

In [ ]:
using Dates, Plots             
using Logging, TerminalLoggers
global_logger(TerminalLogger());

In [ ]:
import RetinalChaos: Network, extract_dict, read_JSON
import RetinalChaos: tar_conds, tar_pars

In [ ]:
dt = 10.0; tspan = (0.0, 300e3)
nx = 96; ny = 96;

In [ ]:
SACnet = Network(96, 96; μ = 0.70, gpu = gpu, version = :gACh)
p_dict = read_JSON("params.json");
u_dict = read_JSON("conds.json");
u0 = extract_dict(u_dict, tar_conds, (nx, ny));
p0 = extract_dict(p_dict, tar_pars);

In [ ]:
import RetinalChaos: SDEProblem, noise_2D, SOSRI, solve

In [ ]:
println("Warming up solution")
prob = SDEProblem(SACnet, noise_2D, u0, (0.0, 60e3), p0);
sol = solve(
    prob,
    SOSRI(),
    abstol = 0.2,
    reltol = 2e-2,
    maxiters = 1e7,
    progress = true, 
    save_everystep = false
)

In [ ]:
prob = SDEProblem(prob.f, prob.g, sol[end], (0.0, 60e3), prob.p);
sol = solve(
        prob,
        SOSRI(),
        abstol = 0.2,
        reltol = 2e-2,
        maxiters = 1e7,
        progress = true, 
        saveat = dt,
    )

### [1.5.b] Graphing and visualization

In [ ]:
import RetinalChaos: calculate_threshold, get_timestamps, max_interval_algorithim, extract_burstmap, timescale_analysis

In [ ]:
vm = Array(sol[:,:,1,:]);

In [ ]:
thresh = calculate_threshold(vm)
spike_array = vm .> thresh;

In [ ]:
rand([])

In [ ]:
burst_map = extract_burstmap(spike_array);
@time anim = @animate for i = 1:10:size(sol, 4)
    println("Animating frame $i")
    p = plot(layout = grid(2, 2), size = (800, 800))
    heatmap!(p[1], sol[:, :, 1, i], ratio = :equal, grid = false,
        xaxis = "", yaxis = "", xlims = (0, nx), ylims = (0, ny),
        c = :curl, clims = (-70.0, 0.0),
    )
    heatmap!(p[2], burst_map[:,:,i], ratio = :equal, grid = false,
        xaxis = "", yaxis = "", xlims = (0, nx), ylims = (0, ny),
        c = :grays, clims = (0.0, 1.0),
    )
    heatmap!(p[3], sol[:, :, 3, i], ratio = :equal, grid = false,
        xaxis = "", yaxis = "", xlims = (0, nx), ylims = (0, ny),
        c = :kgy, clims = (0.0, 1.0),
    )
    heatmap!(p[4], sol[:, :, 6, i], ratio = :equal, grid = false,
        xaxis = "", yaxis = "", xlims = (0, nx), ylims = (0, ny),
        c = :bgy, clims = (0.0, 1.0),
    )
    line_plot = plot(vm[])
    end;

In [ ]:
gif(anim, "t_map.gif", fps = 10)

### [1.5.c] Analysis of the wave Data
- The output of the wave we get is very similar to the output of the trace simulation, except it is in 3D (x, y, d(Var)) with respect to time (t)
- Dispatches of count_interval are available to get all the intervals within the grid. 
    - This function however discards information about the x and y location and adds all the intervals to a single list
- Dispatches of get_timestamps are available max_interval_algorithim
    - These return a tuple with (x, y, data) 
    - for get_timestamps, data is the timestamps
    - for max_interval_algorithgim data is the 
        - 1) Burst Timestamps
        - 2) Duration list
        - 3) Spike per burst list
- A dispatch of the timescale analysis is also available, much like the count intervals all spatial information is lost in the process of conducting the timescale analysis. 
    - In order to conducte more precise spike duration analysis a higher resolution needs to be used which can end up very memory consuming. 

In [ ]:
#get_timestamps() returns (x, y, timestamps)
timestamp_data = @time get_timestamps(spike_array);

In [ ]:
#max_interval_algorithim() returns (x, y, [burst_timestamps, durations, spikes_per_burst and intervurst interval])
burst_data = max_interval_algorithim(spike_array);

In [ ]:
#This function breaks downt the spike array into bursts according to the max interval algorithim
burst_map = extract_burstmap(spike_array);

In [ ]:
using Statistics

In [ ]:
ts_lattice = timescale_analysis(vm; verbose = 1);

#### Comparing to 1D traces

In [ ]:
import RetinalChaos: T_sde

In [ ]:
p = read_JSON("params.json") |> extract_dict;
u0 = read_JSON("conds.json") |> extract_dict;
dt = 1.0
tspan = (0.0, 300e3);
SDEprob = SDEProblem(T_sde, u0, tspan, p)
println("Time it took to simulate 60s:")
@time SDEsol = solve(SDEprob, SOSRI(), abstol = 2e-2, reltol = 2e-2, maxiters = 1e7, saveat = dt); 
trace = Array(SDEsol)';

In [ ]:
vm_iso = trace[240000:end,1];

In [ ]:
plot(vm_iso, lw = 2.0)
plot!(sol.t, vm[1,1,:], lw = 2.0)

In [ ]:
#Conducting the analysis in a single function you can use the imported function timescale_analysis
ts_iso = timescale_analysis(trace[:,1]; dt = dt, verbose = 1);

In [ ]:
p = plot(layout = grid(3,1))
bar!(p[1], [1, ts_lattice[1]], c = :green, label = :lattice)
bar!(p[1], [2, ts_iso[1]], c = :orange, label = :isolated)
bar!(p[2], [ts_lattice[3], ts_iso[3]], c = [:green, :orange])
bar!(p[3], [ts_lattice[5], ts_iso[5]], c = [:green, :orange])

In [ ]:
 ts_lattice[1]

### [1.5.d] We can do multiple runs of this simulation loop. This allows us to 
- Perform repeated trials
- Alter a variable

I will only be doing one of these however, because these trials are costly. The below code should be performed only on a computer with a capable GPU, otherwise this will take a very long time. I will be altering the amount of acetylcholine released by a single SAC in $\mu M / ms$($\rho$)

In [ ]:
par = :ρ
n_trials = 10
par_range = LinRange(1.0, 10.0, n_trials)

In [ ]:
for val in par_range
    with_logger(TerminalLogger()) do
        #Warmup
        SDE_mat_prob = SDEProblem(SACnet, noise_2D, u0, (0.0, 60e3), p0);
        SDE_mat_sol = solve(
            SDE_mat_prob,
            SOSRI(),
            abstol = 0.2,
            reltol = 2e-2,
            maxiters = 1e7,
            progress = true, 
            saveat = dt,
        )
        #Run the trial
        u0_new = SDE_mat_sol[end]
        SDE_mat_prob = SDEProblem(SACnet, noise_2D, u0_new, tspan, p0);
        SDE_mat_sol = solve(
            SDE_mat_prob,
            SOSRI(),
            abstol = 0.2,
            reltol = 2e-2,
            maxiters = 1e7,
            progress = true, 
            saveat = dt,
        )
        #Perform the Data Analysis TODO
    end
end